# Language Model

首先说明，本笔记基本参考[知乎文章](https://zhuanlan.zhihu.com/p/52061158)

- 标准定义：对于语言序列$w_1, w_2, \ldots, w_n$，语言模型就是计算该序列的概率，即 $P(w_1, w_2, \ldots, w_n)$。
- 从机器学习的角度来看：语言模型是对语句的**概率分布**的建模。
- 通俗解释：判断一个语言序列是否是正常语句，即是否是**人话，**例如$P(how\ are\ you) > P(You\ how\ are)$。

## Statistic Language Model

首先，由chain rule可以得到
$$
P(w_1,w_2,\ldots, w_n) = P(w_1)P(w_2|w_1)\ldots P(w_n|w_{n-1}\ldots w_1)
\tag{1}
$$

在统计语言模型中，使用极大似然估计来计算每个词出现的概率。
$$
\begin{aligned}
P(w_i|w_{i-1}\ldots w_1) 
&= \frac{C(w_1, \ldots, w_{i-1}, w_i)}{\sum_wC(w_1, \ldots, w_{i-1}, w)}\\
&\overset{?}{=} \frac{C(w_1, \ldots, w_{i-1}, w_i)}{C(w_1, \ldots, w_{i-1})}
\end{aligned}
\tag{2}
$$

公式(2)中$C(\cdot)$代表第二个通常是不等的，第二个等式的值是明显小于第一个等式的，但是通常使用第二个等式来计算(是为了LM能够对变长序列进行处理)。

对于任意长的自然语言语句，根据极大似然估计直接计算$P(w_i|w_{i-1}\ldots w_1)$不现实。参数量过大，而且出现数据稀疏。

### N-gram
此时，就要引入Marlov Assumption,即当前词只依赖前$n-1$个词。可以得到
$$
P(w_i|w_{i-1}\ldots w_1)  = P(w_i|w_{i-n+1}, \ldots, w_{i-1})
\tag{3}
$$

那么，根据公式(3)

- N=1 Unigram
  $$
  P(w_1, \ldots, w_n) = \displaystyle\prod^n_{i=1}P(w_1)
  \tag{4}
  $$

- N=2 Bigram
  $$
  P(w_1, \ldots, w_n) = \displaystyle\prod^n_{i=1}P(w_i|w_{i-1})
  \tag{5}
  $$

- N=3 Trigram
  $$
  P(w_1, \ldots, w_n) = \displaystyle\prod^n_{i = 1}P(w_i|w_{i-1}w_{i-2})
  \tag{6}
  $$

其中，当$n \gt 1$时，为了使句首词的条件概率有意义，需要给原序列加上一个或多个**起始符**$<start>$可以说起始符的作用就是为了**表征句首词出现的条件概率**。当然，在一些教程上，句首的条件概率直接使用$P(w_1)$，但是通过参考[HMM](../ml/hmm.ipynb)中参数$\pi$的统计方法。还是认为采用在句首添加$<start>$的方式更为合理。同理还需要在句末添加一个$<end>$，这个结束符的目的是让LM能够处理变长序列。使得模型在L的所有序列上的概率和为1。同时，也是使得公式(2)中第二个等式成立的条件。

当然，我们训练模型的语料库不能穷尽语言中所有的序列，那么总会有在训练语言中没有出现过的grams。这就要涉及到[平滑](../math/smoothing.ipynb)处理以及为登录词的处理。我们可以在语言添加一个$<UNK>$用来代替训练预料中没有出现的词的位置。该处同样经过平滑处理。

## Neural Netword Language Model

可以参考[NNLM](https://www.jianshu.com/p/a02ea64d6459)

通过上述内容可知，语言模型实际上就是在求解已知一个序列的情况下，下一个词出现的概率分布。实际上是将$P(w_i|w_{i-1},\ldots,w_1)$的分布去逼近真实的语言中的分布。

### Based on Forward Neural Network

基于前馈神经网络的语言模型借鉴了N-gram的思想，当前词同样只依赖于前$n-1$个词。其模型训练过程如下。假设词典大小为$|V|$，那么有将前$n-1$个词的$one-hot$向量叠加为一个size为$1\times |V|$的向量。设置隐藏层的参数size为$|V|\times H$,输出层的参数为$H\times|V|$，然后经过softmax激活之后与当前词的$ont-hot$求交叉熵。

**其具体实现可以参考[FNN](../dl/fnn.ipynb)** 

利用神经网络去建模当前词出现的概率与其前 n-1 个词之间的约束关系。很显然这种方式相比 N-gram 具有更好的**泛化能力**，**只要词表征足够好。**从而很大程度地降低了数据稀疏带来的问题。但是这个结构的明显缺点是仅包含了**有限的前文信息**。

### Based on Recurrent Neural Network

由于FNN的诸多限制，后续有人提出了基于RNN的LM，此处仅简单的口述，每一个刻的隐藏状态都蕴含了下一时刻词的概率。其具体计算如下，假设t时刻的状态为$h_t \in \R^H$,经过一个size为$H\times|V|$的矩阵乘法后在将结果使用softmax归一化。得到$t+1$时刻为各个词的条件概率。

## 语言模型的评估

通常使用[Perplexity](perplexity.ipynb)